In [225]:
import xml.etree.ElementTree as ET
import json
tree = ET.parse('lowbridges_citywide_data_71309.kml')
root = tree.getroot()

In [91]:
root[0][6].tag
#root.tag

'{http://earth.google.com/kml/2.0}Folder'

In [224]:
for child in root[0][5][4]:
...     print(child.tag, child.text)

{http://earth.google.com/kml/2.0}name  I 87
{http://earth.google.com/kml/2.0}Snippet None
{http://earth.google.com/kml/2.0}description 
					<table border=1 style="border-collapse:collapse; border-color:#000000;" cellpadding=0 cellspacing=0  width=250 style="FONT-SIZE: 11px; FONT-FAMILY: Verdana, Arial, Helvetica, sans-serif;"><tr><td bgcolor="#C0C0C0" align="right"><font COLOR="#000000"><b>BRIDGE</b></font></td><td bgcolor="#FFFFFF"> <font COLOR="#FF0000"> I 87</font></td></tr><tr><td bgcolor="#C0C0C0" align="right"><font COLOR="#000000"><b>Highway</b></font></td><td bgcolor="#FFFFFF"> <font COLOR="#FF0000">MOSHOLU PARKWAY</font></td></tr><tr><td bgcolor="#C0C0C0" align="right"><font COLOR="#000000"><b>BRIDGE_CLEARANCE_FEET</b></font></td><td bgcolor="#FFFFFF"> <font COLOR="#FF0000">11</font></td></tr><tr><td bgcolor="#C0C0C0" align="right"><font COLOR="#000000"><b>BRIDGE_CLEARANCE_INCHES</b></font></td><td bgcolor="#FFFFFF"> <font COLOR="#FF0000">6</font></td></tr><tr><td bgcolor="#C

In [226]:
bridges = []
road_event_feed_info = {
    "publisher": "NYCDOT",
    "version": "3.0",
    "update_date": "2020-09-29T18:00:00Z",
    "update_frequency": 86400,
    "contact_name": "TBD",
    "contact_email": "TBD@gmail.com",
    "data_sources": [
        {
            "data_source_id": "nyc-bridge-data",
            "organization_name": "NYC DOT",
            "location_method": "channel-device-method",
            "contact_name": "TBD",
            "contact_email": "TBD@gmail.com"
        }
    ]
}
#road_event_features
data_source_id = "nyc-bridge-data"
event_type = "work-zone"
start_date = "2020-01-01T00:00:00Z"
end_date = "2021-12-31T00:00:00Z"
beginning_accuracy = "verified"
ending_accuracy = "estimated"
start_date_accuracy = "verified"
end_date_accuracy = "verified"
direction = "westbound"
vehicle_impact = "all-lanes-open"
restrictions = "reduced-height"
#lane features
order = 1
lane_type = "left-lane"
status = "open"
#lane restriction features
restriction_type = "reduced-height"
restriction_units = "feet"




In [228]:
index = 1
for child in root[0][5].iter('{http://earth.google.com/kml/2.0}Placemark'):
    road_event_id = (child[0].text) + str(index) #road_name + index to create unique id
    data_block = (child[2].text)
    data_block = data_block.replace('''<table border=1 style="border-collapse:collapse; border-color:#000000;" cellpadding=0 cellspacing=0  width=250 style="FONT-SIZE: 11px; FONT-FAMILY: Verdana, Arial, Helvetica, sans-serif;">''','<table>')
    data_block = ET.fromstring(data_block)
    road_name = data_block[1][1][0].text #impacted road
    #height restriction in feet
    restriction_value = round(int(data_block[2][1][0].text) + \
                              int(data_block[3][1][0].text)/12,2)
    coords_text = child[4][2].text.split(",")#geospatial coords
    float_coords = [round(float(coords),5) for coords in coords_text]
    dummy_coords = [float_coords[0]+.00001,float_coords[1]+.00001,float_coords[2]]
    dummy_coords = [round(coords,5) for coords in dummy_coords]
    coordinates = [float_coords,dummy_coords] #adds extra dummy value to make Multipoint
    road_event_properties = {
        "data_source_id" : data_source_id,
        "road_event_id" : road_event_id,
        "event_type" : event_type,
        "start_date" : start_date,
        "end_date" : end_date,
        "beginning_accuracy": beginning_accuracy,
        "ending_accuracy": ending_accuracy,
        "start_date_accuracy" : start_date_accuracy,
        "end_date_accuracy" : end_date_accuracy,
        "road_name" : road_name,
        "vehicle_impact" : vehicle_impact,
        "direction" : direction,
        "lanes" : [
            {
                "order": order,
                "status": status,
                "type": lane_type,
                "restrictions":[
                    {
                        "restriction_type" : restriction_type,
                        "restriction_value" : restriction_value, 
                        "restriction_units" : restriction_units
                    }
                ]
            }
        ]
    }
    road_event_geometry = {
        "type" : "MultiPoint",
        "coordinates": coordinates
    }
    road_event = {
        "type" : "Feature",
        "properties" : road_event_properties,
        "geometry" : road_event_geometry
    }
    bridges.append(road_event)
    index = index + 1

In [229]:
data_feed = {
    "road_event_feed_info": road_event_feed_info,
    "type": "FeatureCollection",
    "features": bridges
}


In [190]:
print(data_feed)

{'road_event_feed_info': {'publisher': 'NYCDOT', 'version': '3.0', 'update_date': '2020-09-29T18:00:00Z', 'update_frequency': 86400, 'contact_name': 'TBD', 'contact_email': 'TBD@gmail.com', 'data_sources': [{'data_source_id': 'nyc-bridge-data', 'organization_name': 'NYC DOT', 'location_method': 'channel-device-method', 'contact_name': 'TBD', 'contact_email': 'TBD@gmail.com'}]}, 'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'data_source_id': 'nyc-bridge-data', 'road_event_id': ' I 871', 'event_type': 'work-zone', 'start_date': '2020-01-01T00:00:00Z', 'end_date': '2021-12-31T00:00:00Z', 'start_date_accuracy': 'verified', 'end_date_accuracy': 'verified', 'road_name': 'MOSHOLU PARKWAY', 'direction': 'westbound', 'lanes': [{'order': 1, 'status': 'open', 'type': 'left-lane', 'restrictions': [{'restriction_type': 'reduced-height', 'restriction_value': 11.5, 'restriction_units': 'feet'}]}]}, 'geometry': {'type': 'Multipoint', 'coordinates': [['-73.8875379562,40.8

In [230]:

with open('NYCBridgeWZDx.json', 'w') as outfile:
    json.dump(data_feed, outfile, indent = 4)